In [43]:
import tkinter as tk
from tkinter import ttk, messagebox
from PIL import ImageTk, Image
import requests
from io import BytesIO


class User:
    def __init__(self, username, password, role="user"):
        self.username = username
        self.password = password
        self.role = role


class Ebook:
    def __init__(self, eid, title, author, genre, num_total, num_available, availability=True, cover_image_url=None):
        self.eid = eid
        self.title = title
        self.author = author
        self.genre = genre
        self.num_total = num_total
        self.num_available = num_available
        self.availability = availability
        self.cover_image_url = cover_image_url
        self.reviews = []
        self.ratings = []

    def __str__(self):
        return f"{self.eid:<5} {self.title:<40} {self.author:<30} {self.genre:<20} {self.num_available}/{self.num_total} available"

    def borrow(self):
        if self.num_available > 0:
            self.num_available -= 1
            if self.num_available == 0:
                self.availability = False  # Update availability status when all copies are borrowed
            return f"{self.title} is successfully borrowed."
        else:
            return "All copies of the ebook are currently borrowed."

    def return_book(self):
        if self.num_available < self.num_total:
            self.num_available += 1
            if self.num_available > 0:
                self.availability = True  # Update availability status when a copy is returned
            return f"{self.title} is successfully returned."
        else:
            return "No borrowed copies of the ebook found."

    def rate(self, rating):
        if 1 <= rating <= 5:
            self.ratings.append(rating)
            return f"Rating of {rating} has been added for {self.title}."
        else:
            return "Invalid rating. Rating must be between 1 and 5."

    def average_rating(self):
        if self.ratings:
            return sum(self.ratings) / len(self.ratings)
        else:
            return None


class Library:
    def __init__(self):
        self.ebooks = []
        self.last_eid = 0  # For generating unique IDs
        self.users = []    # List to store user objects
        self.ratings = {}  # Dictionary to store ebook ratings
        self.borrowed_books = {}  # Dictionary to store borrowed books

        # ebooks initialization, btw the 10 and 5 means the total number of books and the number available
        self.add_ebook(Ebook(self.last_eid + 1, "The Great Gatsby", "F. Scott Fitzgerald", "Classic", 10, 1, cover_image_url="https://image.tmdb.org/t/p/original/uCEoQhoRF4EWQnniZRGzDFHuITs.jpg"))
        self.add_ebook(Ebook(self.last_eid + 1, "Batman", "Dwayne Johnson", "Horror", 10, 5, cover_image_url="https://d28hgpri8am2if.cloudfront.net/book_images/onix/cvr9781935703013/my-first-batman-book-9781935703013_xlg.jpg"))
        self.add_ebook(Ebook(self.last_eid + 1, "To Kill A Mockingbird", "Harper Lee", "Classic", 10, 5, cover_image_url="https://cdn2.penguin.com.au/covers/original/9781784870799.jpg"))
        self.add_ebook(Ebook(self.last_eid + 1, "1984", "George Orwell", "Dystopian", 10, 5, cover_image_url="https://www.bookgeeks.in/wp-content/uploads/2021/07/1984-by-George-Orwell-1.jpg"))
        self.add_ebook(Ebook(self.last_eid + 1, "Romeo and Juliet", "William Shakespeare", "Classic", 5, 2, cover_image_url="https://d28hgpri8am2if.cloudfront.net/book_images/cvr9781451621709_9781451621709_hr.jpg"))
        self.add_ebook(Ebook(self.last_eid + 1, "Magic Treehouse", "Mary Pope Osborne", "Children", 9, 4, cover_image_url="https://classroomessentials.scholastic.ca/dw/image/v2/AAXY_PRD/on/demandware.static/-/Sites-master-catalog-cec-ca/default/dw12e98dda/products/9781338251043-7.jpg?sw=960&sh=960&sm=fit&sfrm=jpg"))
        self.add_ebook(Ebook(self.last_eid + 1, "Pride and Prejudice", "Jane Austen", "Classic", 8, 2, cover_image_url="https://d28hgpri8am2if.cloudfront.net/book_images/onix/cvr9781471134746/pride-and-prejudice-9781471134746_hr.jpg"))
        self.add_ebook(Ebook(self.last_eid + 1, "Animal Farm", "George Orwell", "Dystopian", 9, 5, cover_image_url="https://pathakshamabesh.com/wp-content/uploads/2020/07/animal-farm-george-orwell1_132823-768x1186.jpg"))
        self.add_ebook(Ebook(self.last_eid + 1, "Emma", "Jane Austen", "Classic", 4, 2, cover_image_url="https://d28hgpri8am2if.cloudfront.net/book_images/onix/cvr9781501137365/emma-9781501137365_hr.jpg"))
        self.add_ebook(Ebook(self.last_eid + 1, "Harry Potter", "J.K. Rowling", "Fantasy", 10, 8, cover_image_url="https://strangerthansf.com/images/rowling-harrypotterandthegobletoffire.jpg"))
        self.add_ebook(Ebook(self.last_eid + 1, "The Cat in the Hat", "Dr. Seuss", "Children", 5, 2, cover_image_url="https://images.thenile.io/r1000/9780008201517.jpg"))
        self.add_ebook(Ebook(self.last_eid + 1, "The Sneetches", "Dr. Seuss", "Children", 4, 1, cover_image_url="https://www.adazing.com/wp-content/uploads/2020/04/dr.-seuss-13-scaled.jpg"))
        self.add_ebook(Ebook(self.last_eid + 1, "The Lorax", "Dr. Seuss", "Children", 6, 3, cover_image_url="https://fthmb.tqn.com/8e5O5Fpt4Bd53hxCpF5onTUBxNk=/1194x1656/filters:fill(auto,1)/lorax-597f13009abed50010c7da1d.jpg"))
        self.add_ebook(Ebook(self.last_eid + 1, "Hop on Pop", "Dr. Seuss", "Children", 8, 2, cover_image_url="https://www.theepochtimes.com/assets/uploads/2016/02/22/HopOnPop9780394800295.jpg"))
        self.add_ebook(Ebook(self.last_eid + 1, "Scrambled Egg Super", "Dr. Seuss", "Children", 3, 2, cover_image_url="https://www.giantfreakinrobot.com/wp-content/uploads/2021/03/scrambled-1494x2048.jpg"))
        self.add_ebook(Ebook(self.last_eid + 1, "What Pet Should I Get?", "Dr. Seuss", "Children", 2, 1, cover_image_url="https://images-na.ssl-images-amazon.com/images/I/91ZJfQFs9CL.jpg"))
        self.add_ebook(Ebook(self.last_eid + 1, "Matilda", "Roald Dahl", "Children", 3, 1, cover_image_url="https://i.thenile.io/r1000/9780142410370.jpg?r=5eefbb1443735"))
        self.add_ebook(Ebook(self.last_eid + 1, "The Family with the Red Roofs", "Enid Blyton", "Children", 4, 1, cover_image_url="https://i.pinimg.com/originals/83/f0/12/83f0121f820f740324483ec5dde99827.jpg"))
        self.add_ebook(Ebook(self.last_eid + 1, "Happy Hours", "Enid Blyton", "Children", 5, 2, cover_image_url="https://d3525k1ryd2155.cloudfront.net/h/496/347/1032347496.0.x.2.jpg"))
        self.add_ebook(Ebook(self.last_eid + 1, "The Famous Five", "Enid Blyton", "Children", 5, 2, cover_image_url="https://hachette.imgix.net/books/9780340681190.jpg?auto=compress,format"))

    def add_ebook(self, ebook):
        self.last_eid += 1
        ebook.eid = self.last_eid
        self.ebooks.append(ebook)
        self.ratings[ebook.eid] = []  # Initialize ratings for this ebook

    def add_user(self, user):
        self.users.append(user)

    def authenticate_user(self, username, password):
        for user in self.users:
            if user.username == username and user.password == password:
                return user
        return None

    def search_by_title(self, title):
        found_books = []
        for ebook in self.ebooks:
            if title.lower() in ebook.title.lower():
                found_books.append(ebook)
        return found_books

    def display_all_books(self):
        return self.ebooks

    def calculate_average_rating(self, eid):
        ratings = self.ratings.get(eid)
        if ratings:
            return sum(ratings) / len(ratings)
        else:
            return None

    def borrow_ebook(self, user, eid):
        ebook = next((e for e in self.ebooks if e.eid == eid), None)
        if ebook:
            if ebook.availability:
                self.borrowed_books[eid] = user
                ebook.borrow()
                return f"{ebook.title} is successfully borrowed."
            else:
                return "All copies of the ebook are currently borrowed."
        else:
            return "Ebook not found in the library."

    def return_ebook(self, user, eid):
        if eid in self.borrowed_books:
            if self.borrowed_books[eid] == user:
                del self.borrowed_books[eid]
                ebooks = [ebook for ebook in self.ebooks if ebook.eid == eid]
                if ebooks:
                    ebook = ebooks[0]
                    ebook.return_book()
                    return f"{ebook.title} is successfully returned."
                else:
                    return "Ebook not found in the library."
            else:
                return "You cannot return a book that you haven't borrowed."
        else:
            return "No borrowed copies of the ebook found."

    def rate_ebook(self, user, eid, rating):
        if 1 <= rating <= 5:
            ebook = [ebook for ebook in self.ebooks if ebook.eid == eid]
            if ebook:
                ebook[0].rate(rating)
                self.ratings[eid].append(rating)  # Store the rating for this ebook
                return f"Rating of {rating} has been added for ebook ID {eid}."
            else:
                return "Ebook not found in the library."
        else:
            return "Invalid rating. Rating must be between 1 and 5."


class EbookApp:
    def __init__(self, master):
        self.master = master
        self.master.title("Ebook Library")
        self.master.geometry("1000x800")
        self.library = Library()
        self.logged_in_user = None
        self.create_widgets()
        self.style = ttk.Style()
        self.style.configure('Title.TLabel', font=('Arial', 20, 'bold'))

    def create_widgets(self):
        self.welcome_label = ttk.Label(self.master, text="Welcome To My Amazing Ebook Library!", style="Title.TLabel")
        self.welcome_label.pack(pady=20)

        self.login_button = ttk.Button(self.master, text="Login", command=self.show_login)
        self.login_button.pack(pady=20)

        self.create_account_button = ttk.Button(self.master, text="Create Account", command=self.show_create_account)
        self.create_account_button.pack()

    def show_login(self):
        login_window = tk.Toplevel(self.master)
        login_window.title("Login")
        login_window.geometry("400x300")
        
        ttk.Label(login_window, text="Username:", style="Title.TLabel").pack()
        self.entry_username = ttk.Entry(login_window)
        self.entry_username.pack()

        ttk.Label(login_window, text="Password:", style="Title.TLabel").pack()
        self.entry_password = ttk.Entry(login_window, show="*")
        self.entry_password.pack()

        ttk.Button(login_window, text="Login", command=self.login).pack()

    def show_create_account(self):
        create_account_window = tk.Toplevel(self.master)
        create_account_window.title("Create Account")
        create_account_window.geometry("400x300")


        ttk.Label(create_account_window, text="Username:", style="Title.TLabel").pack()
        self.entry_username = ttk.Entry(create_account_window)
        self.entry_username.pack()

        ttk.Label(create_account_window, text="Password:", style="Title.TLabel").pack()
        self.entry_password = ttk.Entry(create_account_window, show="*")
        self.entry_password.pack()

        ttk.Button(create_account_window, text="Create Account", command=self.create_account).pack()

    def login(self):
        username = self.entry_username.get()
        password = self.entry_password.get()
        user = self.library.authenticate_user(username, password)
        if user:
            self.logged_in_user = user
            self.show_menu()
        else:
            messagebox.showerror("Error", "Invalid username or password")

    def create_account(self):
        username = self.entry_username.get()
        password = self.entry_password.get()
        new_user = User(username, password)
        self.library.add_user(new_user)
        messagebox.showinfo("Success", "Account created successfully")

    def show_menu(self):
        menu_window = tk.Toplevel(self.master)
        menu_window.title("Menu")
        menu_window.geometry("400x300")

        ttk.Label(menu_window, text=f"Welcome, {self.logged_in_user.username}!", style="Title.TLabel").pack()

        ttk.Button(menu_window, text="Search ebooks by title", command=self.show_search_by_title).pack()
        ttk.Button(menu_window, text="All Ebooks", command=self.show_all_books).pack()
        ttk.Button(menu_window, text="Borrow an ebook", command=self.show_borrow_ebook).pack()
        ttk.Button(menu_window, text="Return an ebook", command=self.show_return_ebook).pack()
        ttk.Button(menu_window, text="Rate an ebook", command=self.show_rate_ebook).pack()

    def show_search_by_title(self):
        search_window = tk.Toplevel(self.master)
        search_window.title("Search by Title")
        search_window.geometry("400x350")

        ttk.Label(search_window, text="Enter title:", style="Title.TLabel").pack()
        self.entry_title = ttk.Entry(search_window)
        self.entry_title.pack()

        ttk.Button(search_window, text="Search", command=self.perform_title_search).pack()
        ttk.Button(search_window, text="Back", command=search_window.destroy).pack(pady=10)

    def perform_title_search(self):
        title = self.entry_title.get()
        result = self.library.search_by_title(title)
        result_window = None
        if result:
            result_window = tk.Toplevel(self.master)
            result_window.title("Search Results")
            result_window.geometry("400x300")

            ttk.Label(result_window, text="Search Results:", style="Title.TLabel").pack()

            result_listbox = tk.Listbox(result_window)
            result_listbox.pack(expand=True, fill="both")

            for ebook in result:
                result_listbox.insert(tk.END, ebook)
        else:
            messagebox.showinfo("Info", "No ebooks found.")

    def show_all_books(self):
        books_window = tk.Toplevel(self.master)
        books_window.title("All Books")
        books_window.geometry("1000x800")
    
        canvas = tk.Canvas(books_window)
        canvas.pack(side="left", fill="both", expand=True)
    
        scrollbar = ttk.Scrollbar(books_window, orient="vertical", command=canvas.yview)
        scrollbar.pack(side="right", fill="y")
    
        canvas.configure(yscrollcommand=scrollbar.set)
    
        frame = tk.Frame(canvas)
        canvas.create_window((0, 0), window=frame, anchor="nw")
    
        ttk.Label(frame, text="All Books:", style="Title.TLabel").pack()
    
        books = self.library.display_all_books()
        for ebook in books:
            subframe = tk.Frame(frame)
            subframe.pack(fill="both", padx=10, pady=5)

            if ebook.cover_image_url:
                response = requests.get(ebook.cover_image_url)
                img_data = response.content
                img = Image.open(BytesIO(img_data))
                img = img.resize((100, 150))
                photo = ImageTk.PhotoImage(img)
                label_image = tk.Label(subframe, image=photo)
                label_image.image = photo
                label_image.pack(side="left")

            details_text = f"ID: {ebook.eid}\nTitle: {ebook.title}\nAuthor: {ebook.author}\nGenre: {ebook.genre}\nAvailability: {ebook.num_available}/{ebook.num_total}"
            tk.Label(subframe, text=details_text).pack(side="left", padx=10)

            average_rating = self.library.calculate_average_rating(ebook.eid)
            if average_rating is not None:
                tk.Label(subframe, text=f"Average Rating: {average_rating:.2f}").pack(side="left", padx=10)
            else:
                tk.Label(subframe, text="Average Rating: N/A").pack(side="left", padx=10)

            ttk.Button(subframe, text="Borrow", command=lambda eid=ebook.eid: self.borrow_ebook(eid)).pack(side="right", padx=10)

        ttk.Button(books_window, text="Back to Main Menu", command=books_window.destroy).pack(pady=10)

        canvas.bind("<Configure>", lambda event, canvas=canvas: canvas.configure(scrollregion=canvas.bbox("all")))
    

    def borrow_ebook(self, eid):
        result = self.library.borrow_ebook(self.logged_in_user, eid)
        messagebox.showinfo("Borrow Ebook", result)

    def show_borrow_ebook(self):
        borrow_window = tk.Toplevel(self.master)
        borrow_window.title("Borrow Ebook")
        borrow_window.geometry("400x300")

        ttk.Label(borrow_window, text="Enter Ebook ID:", style="Title.TLabel").pack()
        self.entry_eid = ttk.Entry(borrow_window)
        self.entry_eid.pack()

        ttk.Button(borrow_window, text="Borrow", command=self.borrow).pack()
        ttk.Button(borrow_window, text="Back", command=borrow_window.destroy).pack(pady=10)

    def borrow(self):
        eid = int(self.entry_eid.get())
        result = self.library.borrow_ebook(self.logged_in_user, eid)
        messagebox.showinfo("Borrow Ebook", result)

    def show_return_ebook(self):
        return_window = tk.Toplevel(self.master)
        return_window.title("Return Ebook")
        return_window.geometry("400x300")

        ttk.Label(return_window, text="Enter Ebook ID:", style="Title.TLabel").pack()
        self.entry_eid = ttk.Entry(return_window)
        self.entry_eid.pack()

        ttk.Button(return_window, text="Return", command=self.return_ebook).pack()
        ttk.Button(return_window, text="Back", command=return_window.destroy).pack(pady=10)

    def return_ebook(self):
        eid = int(self.entry_eid.get())
        result = self.library.return_ebook(self.logged_in_user, eid)
        messagebox.showinfo("Return Ebook", result)

    def show_rate_ebook(self):
        rate_window = tk.Toplevel(self.master)
        rate_window.title("Rate Ebook")
        rate_window.geometry("400x300")

        ttk.Label(rate_window, text="Enter Ebook ID:", style="Title.TLabel").pack()
        self.entry_eid = ttk.Entry(rate_window)
        self.entry_eid.pack()

        ttk.Label(rate_window, text="Enter Rating (1-5):", style="Title.TLabel").pack()
        self.entry_rating = ttk.Entry(rate_window)
        self.entry_rating.pack()

        ttk.Button(rate_window, text="Rate", command=self.rate_ebook).pack()
        ttk.Button(rate_window, text="Back", command=rate_window.destroy).pack(pady=10)

    def rate_ebook(self):
        eid = int(self.entry_eid.get())
        rating = int(self.entry_rating.get())
        result = self.library.rate_ebook(self.logged_in_user, eid, rating)
        messagebox.showinfo("Rate Ebook", result)


root = tk.Tk()
app = EbookApp(root)
root.mainloop()
